In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

from utils import mod_df, inverse_mod_X, inverse_mod_y
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, normalize
# from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

In [ ]:
train_df = pd.read_csv(os.path.join('data', 'unionTrain.csv'))
test_df = pd.read_csv(os.path.join('data', 'unionTest.csv'))
print(f"Length Train: {len(train_df)}")
print(f"Length Test: {len(test_df)}")

In [ ]:
print(len(train_df))
train_df.head()

In [ ]:
train_df['sequence_id'].head(10).to_list()

In [ ]:
X_train, y_train = train_df.drop(['Severity', 'sequence_id'], axis=1), train_df['Severity']
X_test, y_test = test_df.drop(['Severity', 'sequence_id'], axis=1), test_df['Severity']

In [ ]:
def modify_X(x):
    if type(x).__module__ != np.__name__:
        result = x.to_numpy()
    else:
        result = x.copy()
    result = result.reshape(-1, 854, 2*25)
    return result
X_train_modified = modify_X(X_train)
X_train_modified.shape

In [ ]:
def plot_sample_1():
    for i in range(1,5):
        print(f"Index: {i}, Severity: {y_train[i]}")
        for j in range(25):
            sample = X_train_modified[i,j,:]
            plt.plot(sample)
            plt.show()

def plot_sample_2():
    for i in range(7):
        print(f"Index: {i}, Severity: {y_train[i]}")
        for j in range(0, 2, 2):
            sample_xx = X_train_modified[i,:,j]
            sample_yy = X_train_modified[i,:,j+1]
            plt.plot(sample_xx)
            plt.plot(sample_yy)
            plt.show()
            
def plot_sample_3():
    for i in range(1):
        print(f"Index: {i}, Severity: {y_train[i]}")
        for j in range(0, 2, 2):
            LENGTH = 20
            sample_xx = X_train_modified[i,:,j][:LENGTH]
            sample_yy = X_train_modified[i,:,j+1][:LENGTH]
            # print(sample_xx.shape)
            fig = plt.figure()
            ax = plt.axes(projection='3d')
            zline = np.arange(0, min(sample_xx.shape[0], LENGTH), 1)
            xline = np.sin(sample_xx)
            yline = np.cos(sample_yy)
            ax.plot3D(xline, yline, zline, 'green')

        
# plot_sample_1()
plot_sample_2()
# plot_sample_3()

In [ ]:
# for i in range(5):
#     print(f"Index: {i}, Severity: {y_train[i]}")
#     plt.plot(X_train_modded[['18X', '17X']][i*854:i*854+854])
#     plt.show()